In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.realpath('__file__')) + '/../../../')
from acaisdk.file import File
from acaisdk.project import Project
from acaisdk.fileset import FileSet
from acaisdk.job import *
from acaisdk.meta import *
from acaisdk.utils import utils
from acaisdk import credentials

workspace = os.path.dirname(os.path.realpath('__file__'))
utils.DEBUG = True  # print debug messages. Calls are successful as long as no exception is thrown.

In [11]:
# Make your changes here
project_id = "ray-profile-job-2"
root_token = 'EmDlCTBF1ppONSciYVd03M9xkmF6hFqW'
project_admin = 'zhengzhe'
user = 'zhengzhe'

In [12]:
# Create project and user
r = Project.create_project(project_id, root_token, project_admin)
r = Project.create_user(project_id, r['project_admin_token'], user)  # the new user is logged in automatically.

Running request: 35.237.138.197 80 credential create_project
POST data {"project_id": "ray-profile-job-2", "admin_token": "EmDlCTBF1ppONSciYVd03M9xkmF6hFqW", "project_admin_name": "zhengzhe"}
Running request: 35.237.138.197 80 credential create_user
POST data {"project_id": "ray-profile-job-2", "admin_token": "ljIdh5hzfiqLbanwUuqGIf69z3uxoF6g", "user_name": "zhengzhe"}
Logged in with token TTNhsyg3StVMTgn84hdN0UBR3unh8e5s


In [2]:
credentials.login('TTNhsyg3StVMTgn84hdN0UBR3unh8e5s')

In [4]:
# Upload code
train_code = os.path.join(workspace, 'example.zip')
# eval_code = os.path.join(workspace, 'demo/eval_script.zip')
File.upload({train_code: 'ray_profiler_train_scripts.zip'})

Running request: 192.168.99.105 7445 storage start_file_upload_session
POST data {"paths": ["ray_profiler_train_scripts.zip"], "token": "l72GxEK8rNBTBQ8BI1vO9jaF02nYJz8k"}
[('/Users/mangod666/Google Drive/Academics/Fall 2020/Capstone/ACAI/acaisdk/acaisdk/example/ray-profiler-example/example.zip', 'ray_profiler_train_scripts.zip')]
r =  <Response [200]>
Uploaded /Users/mangod666/Google Drive/Academics/Fall 2020/Capstone/ACAI/acaisdk/acaisdk/example/ray-profiler-example/example.zip to ray_profiler_train_scripts.zip
Running request: 192.168.99.105 7445 storage poll_file_upload_session
GET query {"session_id": 9, "token": "l72GxEK8rNBTBQ8BI1vO9jaF02nYJz8k"}
Running request: 192.168.99.105 7445 storage poll_file_upload_session
GET query {"session_id": 9, "token": "l72GxEK8rNBTBQ8BI1vO9jaF02nYJz8k"}
Running request: 192.168.99.105 7445 storage poll_file_upload_session
GET query {"session_id": 9, "token": "l72GxEK8rNBTBQ8BI1vO9jaF02nYJz8k"}
Running request: 192.168.99.105 7445 storage finish_

[('/Users/mangod666/Google Drive/Academics/Fall 2020/Capstone/ACAI/acaisdk/acaisdk/example/ray-profiler-example/example.zip',
  'ray_profiler_train_scripts.zip:1')]

In [55]:
# Upload dummy input files and create a new file set on the fly
input_dir = os.path.join(workspace, 'dataset/')
File.convert_to_file_mapping([input_dir], 'ray_dataset/')\
    .files_to_upload\
    .upload()\
    .as_new_file_set('ray_test_fs')

Nothing to upload
Running request: 192.168.99.105 7445 storage create_file_set
POST data {"name": "ray_test_fs", "files": [], "token": "b3Kbhd8ICdREPE4uMmEwEjvICKQrq4nr"}


{'id': 'ray_test_fs:1', 'files': []}

In [12]:
# Run a profiling job
job_setting = {
    "command_template": "ray start --node-ip-address=$MY_POD_IP --num-cpus=0 --address=$RAY_HEAD_SERVICE_HOST:$RAY_HEAD_SERVICE_PORT_REDIS_PRIMARY --object-manager-port=12345 --node-manager-port=12346 && mkdir ray_output && python example.py | tee ./ray_output/output.txt",
    "container_image": "rayproject/autoscaler",
    'input_file_set': 'ray_test_fs', # not used since this example does not need an input dataset
    'code': 'ray_profiler_train_scripts.zip',
    'name': 'ray_profiling_test',
    'framework': 'ray',
    'nnode_space': [1,3]
}

result = {}
j = ProfilingJob().with_attributes(job_setting)


Running request: 192.168.99.105 7445 storage resolve_file_set
GET query {"vague_name": "ray_test_fs", "token": "l72GxEK8rNBTBQ8BI1vO9jaF02nYJz8k"}
Running request: 192.168.99.105 7445 job_registry new_job
POST data {"name": "ray_test_job", "input_file_set": "ray_test_fs:1", "output_path": "./ray_output/", "code": "ray_profiler_train_scripts.zip", "command": "ray start --node-ip-address=$MY_POD_IP --num-cpus=0 --address=$RAY_HEAD_SERVICE_HOST:$RAY_HEAD_SERVICE_PORT_REDIS_PRIMARY --object-manager-port=12345 --node-manager-port=12346 && mkdir ray_output && python example.py | tee ./ray_output/output.txt", "container_image": "rayproject/autoscaler", "description": "sample ray job with 4 nodes (1 head + 3 workers)", "v_cpu": "100m", "gpu": "0", "memory": "512Mi", "job_status": null, "nnode": 3, "framework": "ray", "token": "l72GxEK8rNBTBQ8BI1vO9jaF02nYJz8k"}
{'status': {'message': 'launching'}, 'job': {'name': 'ray_test_job', 'code': 'ray_profiler_train_scripts.zip', 'command': 'ray start -

In [16]:
# this will populate the result dict with the response, you can also get the response by assigning the return value to a variable
j.profile(result)

Running request: 192.168.1.72 7445 profiler register_template
POST data {"name": "ray_profiling_test", "input_file_set": "ray_test_fs", "code": "ray_profiler_train_scripts.zip", "command_template": "ray start --node-ip-address=$MY_POD_IP --num-cpus=0 --address=$RAY_HEAD_SERVICE_HOST:$RAY_HEAD_SERVICE_PORT_REDIS_PRIMARY --object-manager-port=12345 --node-manager-port=12346 && mkdir ray_output && python example.py | tee ./ray_output/output.txt", "container_image": "rayproject/autoscaler", "nnode_space": [1, 3], "framework": "ray", "token": "qjbYDwKhg8l4OzJymKGy2ruHOodiFgsg"}
{'id': 6, 'name': 'ray_profiling_test', 'code': 'ray_profiler_train_scripts.zip', 'allOptionsSer': '[]', 'state': 'submitted', 'input_file_set': 'ray_test_fs', 'command_template': 'ray start --node-ip-address=$MY_POD_IP --num-cpus=0 --address=$RAY_HEAD_SERVICE_HOST:$RAY_HEAD_SERVICE_PORT_REDIS_PRIMARY --object-manager-port=12345 --node-manager-port=12346 && mkdir ray_output && python example.py | tee ./ray_output/out

{'id': 6,
 'name': 'ray_profiling_test',
 'code': 'ray_profiler_train_scripts.zip',
 'allOptionsSer': '[]',
 'state': 'submitted',
 'input_file_set': 'ray_test_fs',
 'command_template': 'ray start --node-ip-address=$MY_POD_IP --num-cpus=0 --address=$RAY_HEAD_SERVICE_HOST:$RAY_HEAD_SERVICE_PORT_REDIS_PRIMARY --object-manager-port=12345 --node-manager-port=12346 && mkdir ray_output && python example.py | tee ./ray_output/output.txt',
 'memory_space': [2048],
 'cpu_space': [0.5],
 'nnode_space': [1, 3],
 'user_id': 38,
 'project_id': 'execution_test',
 'container_image': 'rayproject/autoscaler',
 'model_name': None,
 'framework': 'ray',
 'model_id': None,
 'spawned_jobs': 0}

In [16]:
# Upload dummy input files and create a new file set on the fly
input_dir = os.path.join(workspace, 'dataset/')
File.convert_to_file_mapping([input_dir], 'ray_dataset/')\
    .files_to_upload\
    .upload()\
    .as_new_file_set('ray_test_fs')

Nothing to upload
Running request: 35.237.138.197 80 storage create_file_set
POST data {"name": "ray_test_fs", "files": [], "token": "TTNhsyg3StVMTgn84hdN0UBR3unh8e5s"}


{'id': 'ray_test_fs:1', 'files': []}

In [14]:
# Upload code
train_code = os.path.join(workspace, 'code/ray_sgd_example.zip')
# eval_code = os.path.join(workspace, 'demo/eval_script.zip')
File.upload({train_code: 'ray_sgd_example.zip'})

Running request: 35.237.138.197 80 storage start_file_upload_session
POST data {"paths": ["ray_sgd_example.zip"], "token": "fjhZ6Ry3daKKR7uRFoNKiyLVyOK9HPiZ"}
[('/Users/mangod666/Google Drive/Academics/Fall 2020/Capstone/ACAI/acaisdk/acaisdk/example/ray-profiler-example/code/ray_sgd_example.zip', 'ray_sgd_example.zip')]
r =  <Response [200]>
Uploaded /Users/mangod666/Google Drive/Academics/Fall 2020/Capstone/ACAI/acaisdk/acaisdk/example/ray-profiler-example/code/ray_sgd_example.zip to ray_sgd_example.zip
Running request: 35.237.138.197 80 storage poll_file_upload_session
GET query {"session_id": 4781, "token": "fjhZ6Ry3daKKR7uRFoNKiyLVyOK9HPiZ"}
Running request: 35.237.138.197 80 storage poll_file_upload_session
GET query {"session_id": 4781, "token": "fjhZ6Ry3daKKR7uRFoNKiyLVyOK9HPiZ"}
Running request: 35.237.138.197 80 storage poll_file_upload_session
GET query {"session_id": 4781, "token": "fjhZ6Ry3daKKR7uRFoNKiyLVyOK9HPiZ"}
Running request: 35.237.138.197 80 storage finish_file_upl

[('/Users/mangod666/Google Drive/Academics/Fall 2020/Capstone/ACAI/acaisdk/acaisdk/example/ray-profiler-example/code/ray_sgd_example.zip',
  'ray_sgd_example.zip:1')]

In [28]:
# Run a training job, it takes ~3 mins to finish
job_setting = {
    "v_cpu": "200m",
    "memory": "512Mi",
    "gpu": "0",
    "nnode": 1, # the number of workers in addition to the head node
    "framework": "ray",
    "command": "ray start --node-ip-address=$MY_POD_IP --num-cpus=0 --address=$RAY_HEAD_SERVICE_HOST:$RAY_HEAD_SERVICE_PORT_REDIS_PRIMARY --object-manager-port=12345 --node-manager-port=12346 && mkdir ray_output && python ray_sgd_example.py | tee ./ray_output/output.txt",
    "container_image": "rayproject/autoscaler",
    'input_file_set': 'ray_test_fs', # not used since this example does not need an input dataset
    'output_path': './ray_output/', # necessary to have a parent folder
    'code': 'ray_sgd_example.zip',
    'description': 'sample ray sgd job with 2 nodes (1 head + 2 workers)',
    'name': 'ray_test_job'
}

train_job = Job().with_attributes(job_setting).run()

Running request: 35.237.138.197 80 storage resolve_file_set
GET query {"vague_name": "ray_test_fs", "token": "fjhZ6Ry3daKKR7uRFoNKiyLVyOK9HPiZ"}
Running request: 35.237.138.197 80 job_registry new_job
POST data {"name": "ray_test_job", "input_file_set": "ray_test_fs:1", "output_path": "./ray_output/", "code": "ray_sgd_example.zip", "command": "ray start --node-ip-address=$MY_POD_IP --num-cpus=0 --address=$RAY_HEAD_SERVICE_HOST:$RAY_HEAD_SERVICE_PORT_REDIS_PRIMARY --object-manager-port=12345 --node-manager-port=12346 && mkdir ray_output && python ray_sgd_example.py | tee ./ray_output/output.txt", "container_image": "rayproject/autoscaler", "description": "sample ray sgd job with 2 nodes (1 head + 2 workers)", "v_cpu": "1", "gpu": "0", "memory": "512Mi", "job_status": null, "nnode": 2, "framework": "ray", "token": "fjhZ6Ry3daKKR7uRFoNKiyLVyOK9HPiZ"}
{'status': {'message': 'launching'}, 'job': {'name': 'ray_test_job', 'code': 'ray_sgd_example.zip', 'command': 'ray start --node-ip-address=

In [14]:
# Upload code
train_code = os.path.join(workspace, 'code/mnist_ray_sgd.zip')
# eval_code = os.path.join(workspace, 'demo/eval_script.zip')
File.upload({train_code: 'mnist_ray_sgd.zip'})

Running request: 35.237.138.197 80 storage start_file_upload_session
POST data {"paths": ["mnist_ray_sgd.zip"], "token": "TTNhsyg3StVMTgn84hdN0UBR3unh8e5s"}
[('/Users/mangod666/Google Drive/Academics/Fall 2020/Capstone/ACAI/acaisdk/acaisdk/example/ray-profiler-example/code/mnist_ray_sgd.zip', 'mnist_ray_sgd.zip')]
r =  <Response [200]>
Uploaded /Users/mangod666/Google Drive/Academics/Fall 2020/Capstone/ACAI/acaisdk/acaisdk/example/ray-profiler-example/code/mnist_ray_sgd.zip to mnist_ray_sgd.zip
Running request: 35.237.138.197 80 storage poll_file_upload_session
GET query {"session_id": 5858, "token": "TTNhsyg3StVMTgn84hdN0UBR3unh8e5s"}
Running request: 35.237.138.197 80 storage poll_file_upload_session
GET query {"session_id": 5858, "token": "TTNhsyg3StVMTgn84hdN0UBR3unh8e5s"}
Running request: 35.237.138.197 80 storage poll_file_upload_session
GET query {"session_id": 5858, "token": "TTNhsyg3StVMTgn84hdN0UBR3unh8e5s"}
Running request: 35.237.138.197 80 storage finish_file_upload_sessio

[('/Users/mangod666/Google Drive/Academics/Fall 2020/Capstone/ACAI/acaisdk/acaisdk/example/ray-profiler-example/code/mnist_ray_sgd.zip',
  'mnist_ray_sgd.zip:1')]

In [3]:
File.list_dir('/')
# File.download({'/mnist_ray_sgd.zip': '.'})

Running request: 35.237.138.197 80 storage list_directory
GET query {"directory_path": "/", "token": "TTNhsyg3StVMTgn84hdN0UBR3unh8e5s"}


[{'path': 'mnist_ray_sgd.zip', 'version': 1, 'dir': False, 'is_dir': False}]

In [3]:
job_setting = {
    "v_cpu": "100m",
    "memory": "512Mi",
    "gpu": "0",
    "nnode": 2, # the number of workers in addition to the head node
    "framework": "ray",
    "command": "ray start --node-ip-address=$MY_POD_IP --num-cpus=0 --address=$RAY_HEAD_SERVICE_HOST:$RAY_HEAD_SERVICE_PORT_REDIS_PRIMARY --object-manager-port=12345 --node-manager-port=12346 && mkdir ray_output && python mnist_ray_sgd.py | tee ./ray_output/output.txt",
    "container_image": "rayproject/autoscaler",
    'input_file_set': 'ray_test_fs', # not used since this example does not need an input dataset
    'output_path': './ray_output/', # necessary to have a parent folder
    'code': 'mnist_ray_sgd.zip',
    'description': 'sample ray sgd job with 5 nodes (1 head + 4 workers)',
    'name': 'ray_test_job'
}

train_job = Job().with_attributes(job_setting).run()

Running request: 35.237.138.197 80 storage resolve_file_set
GET query {"vague_name": "ray_test_fs", "token": "TTNhsyg3StVMTgn84hdN0UBR3unh8e5s"}
Running request: 35.237.138.197 80 job_registry new_job
POST data {"name": "ray_test_job", "input_file_set": "ray_test_fs:2", "output_path": "./ray_output/", "code": "mnist_ray_sgd.zip", "command": "ray start --node-ip-address=$MY_POD_IP --num-cpus=0 --address=$RAY_HEAD_SERVICE_HOST:$RAY_HEAD_SERVICE_PORT_REDIS_PRIMARY --object-manager-port=12345 --node-manager-port=12346 && mkdir ray_output && python mnist_ray_sgd.py | tee ./ray_output/output.txt", "container_image": "rayproject/autoscaler", "description": "sample ray sgd job with 5 nodes (1 head + 4 workers)", "v_cpu": "100m", "gpu": "512Mi", "memory": "512Mi", "job_status": null, "nnode": 2, "framework": "ray", "token": "TTNhsyg3StVMTgn84hdN0UBR3unh8e5s"}
{'status': None, 'job': {'name': 'ray_test_job', 'code': 'mnist_ray_sgd.zip', 'command': 'ray start --node-ip-address=$MY_POD_IP --num-cp